<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Riley/ETL_Expenditures_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-09 16:39:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-09-09 16:39:43 (11.5 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Expenditures+2021+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Expenditures+2021+(UF).csv"), sep=",", header=True)
df.show()

+--------+--------+------+--------+--------+---------+------+---------+------+--------+------+----------+--------+---------+----------+--------------------+------+-------+-----+-------+---------+---------------+-----+-----+----------+--------+---------+----------------+-----------------+--------+------+-------+
|ELECTION|OFFICECD|CANDID|CANCLASS|CANDLAST|CANDFIRST|CANDMI|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO| INV_DATE|      DATE|                NAME|C_CODE|ORG_IND|STRNO|STRNAME|APARTMENT|           CITY|STATE|  ZIP|PAY_METHOD|    AMNT|PURPOSECD|         PURPOSE|          EXPLAIN|EXEMPTCD|RR_IND|SEG_IND|
+--------+--------+------+--------+--------+---------+------+---------+------+--------+------+----------+--------+---------+----------+--------------------+------+-------+-----+-------+---------+---------------+-----+-----+----------+--------+---------+----------------+-----------------+--------+------+-------+
|    2021|       1|  2445|      NP| Francis|   Quanda|     S|

In [5]:
df.columns

['ELECTION',
 'OFFICECD',
 'CANDID',
 'CANCLASS',
 'CANDLAST',
 'CANDFIRST',
 'CANDMI',
 'COMMITTEE',
 'FILING',
 'SCHEDULE',
 'PAGENO',
 'SEQUENCENO',
 'REFNO',
 'INV_DATE',
 'DATE',
 'NAME',
 'C_CODE',
 'ORG_IND',
 'STRNO',
 'STRNAME',
 'APARTMENT',
 'CITY',
 'STATE',
 'ZIP',
 'PAY_METHOD',
 'AMNT',
 'PURPOSECD',
 'PURPOSE',
 'EXPLAIN',
 'EXEMPTCD',
 'RR_IND',
 'SEG_IND']

In [6]:
# Remove multiple columns
list = ['CANDMI','COMMITTEE',
 'FILING',
 'SCHEDULE',
 'PAGENO',
 'SEQUENCENO',
 'REFNO',
 'INV_DATE',
 'DATE',
 'NAME',
 'ORG_IND',
 'STRNO',
 'STRNAME',
 'APARTMENT',
  'EXPLAIN',
 'EXEMPTCD',
 'RR_IND',
 'SEG_IND',
 'PAY_METHOD'] 
df = df.drop(*list)
df.show()

+--------+--------+------+--------+--------+---------+------+---------------+-----+-----+--------+---------+----------------+
|ELECTION|OFFICECD|CANDID|CANCLASS|CANDLAST|CANDFIRST|C_CODE|           CITY|STATE|  ZIP|    AMNT|PURPOSECD|         PURPOSE|
+--------+--------+------+--------+--------+---------+------+---------------+-----+-----+--------+---------+----------------+
|    2021|       1|  2445|      NP| Francis|   Quanda|  CORP|       Brooklyn|   NY|11201| 2001.16|    RENTO|     Office Rent|
|    2021|      11|  2546|       P|    Yang|   Andrew|  CORP|       Glendale|   NY|11385|   91.57|    OTHER|Other: explntion|
|    2021|      11|   148|       P|Stringer|    Scott|   LLC|       New York|   NY|10006| 3240.75|    RENTO|     Office Rent|
|    2021|      11|   148|       P|Stringer|    Scott|   LLC|       New York|   NY|10006|13240.75|    RENTO|     Office Rent|
|    2021|      11|   148|       P|Stringer|    Scott|   LLC|       New York|   NY|10006| 16762.5|    RENTO|     Offic

In [7]:
# check data types
df.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('CANDID', 'string'),
 ('CANCLASS', 'string'),
 ('CANDLAST', 'string'),
 ('CANDFIRST', 'string'),
 ('C_CODE', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('AMNT', 'string'),
 ('PURPOSECD', 'string'),
 ('PURPOSE', 'string')]

In [8]:
df.columns

['ELECTION',
 'OFFICECD',
 'CANDID',
 'CANCLASS',
 'CANDLAST',
 'CANDFIRST',
 'C_CODE',
 'CITY',
 'STATE',
 'ZIP',
 'AMNT',
 'PURPOSECD',
 'PURPOSE']

In [9]:
#Change column names
df1 = df \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("CANDID", "CandidateID") \
.withColumnRenamed("CANDLAST", "CandidateLastName") \
.withColumnRenamed('CANDFIRST', "CandidateFirstName") \
.withColumnRenamed("C_CODE", "ContributionType") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("PURPOSECD", "PurposeCode") \
.withColumnRenamed("PURPOSE", "Purpose") \
#.withColumnRenamed("PAY_METHOD", "PaymentMethod") 

df1.printSchema()

root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CandidateID: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateLastName: string (nullable = true)
 |-- CandidateFirstName: string (nullable = true)
 |-- ContributionType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- PurposeCode: string (nullable = true)
 |-- Purpose: string (nullable = true)



In [10]:
# changing datatypes of columns
df2 = df1 \
  .withColumn("Election",df1["Election"].cast("Integer")) \
  .withColumn("OFFICECD", df1["OFFICECD"].cast("Integer")) \
  .withColumn("CandidateID", df1["CandidateID"].cast("Integer")) \
  .withColumn("ZipCode", df1["ZipCode"].cast("Integer")) \
  .withColumn("Amount", df1["Amount"].cast("Float")) \
  
df2.printSchema()

root
 |-- Election: integer (nullable = true)
 |-- OFFICECD: integer (nullable = true)
 |-- CandidateID: integer (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateLastName: string (nullable = true)
 |-- CandidateFirstName: string (nullable = true)
 |-- ContributionType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- Amount: float (nullable = true)
 |-- PurposeCode: string (nullable = true)
 |-- Purpose: string (nullable = true)



In [11]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df3 = df2.withColumn("ContributionType", regexp_replace("ContributionType", 'CAN', 'Candidate')) \
  .withColumn('ContributerType', regexp_replace("ContributionType", 'CORP', 'Corporation')) \
  .withColumn('ContributerType', regexp_replace("ContributionType", 'EMPO', 'Labor Union')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'FAM', 'Candidate Family')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'IND', 'Individual')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PCOMZ', 'Party Committee')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'ORG', 'Orgainization')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'LLC', 'Limited Liability Company')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PCOMP', 'Political Action Committee')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'SPO', ' Candidates Spouse')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'OTHR', 'Other')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PCOMC', 'Candidate Committee')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PART', 'Individual')) \


In [14]:
#Change Purpose Code value names
from pyspark.sql.functions import regexp_replace
df4 = df3.withColumn('PurposeCode', regexp_replace('PurposeCode', 'ADVAN', 'Advance Repayment')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'CMAIL', 'Campaign Mail')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'CMISC', 'Miscellaneous')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'CNTRB', 'Political Contributions')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'COMPL', 'Compliance Cost')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'CONSL', 'Campaign Consultants')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'CONSV', 'Constituent Services')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'FUNDR', 'Fundraising')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'INTER', 'Interest Expenses')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'LITER', 'Campaign Literature')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'NQUAL', 'Non Qualified Expenditures')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'OFFCE', 'Office Expenses')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'OTHER', 'Other')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'PETIT', 'Petition Expenses')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'POLLS', 'Polling Costs')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'POSTA', 'Postal Expenses')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'PRINT', 'Printing Expenses')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'PROFL', 'Professional Services')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'RADIO', 'Radio Ads')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'RENTO', 'Office Rent')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'TVADS', 'Television Ads')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'UNKN', 'Unknown')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'VOTER', 'Voter Registration')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'WAGES', 'Campaign Workers Salaries')) \

In [16]:
#Call only Mayor (1) and particaptes (P) within Dataframe
df5 = df4.filter((df4.OFFICECD=="1") & (df4.CANCLASS=="P"))
df5.show()

+--------+--------+-----------+--------+-----------------+------------------+--------------------+----------+-----+-------+------+--------------------+----------------+---------------+
|Election|OFFICECD|CandidateID|CANCLASS|CandidateLastName|CandidateFirstName|    ContributionType|      City|State|ZipCode|Amount|         PurposeCode|         Purpose|ContributerType|
+--------+--------+-----------+--------+-----------------+------------------+--------------------+----------+-----+-------+------+--------------------+----------------+---------------+
|    2021|       1|       1160|       P|         Diaz Jr.|             Ruben|                CORP|  New York|   NY|  10018|1300.0|         Fundraising|     Fundraising|           CORP|
|    2021|       1|       1160|       P|         Diaz Jr.|             Ruben|                CORP|  New York|   NY|  10018| 500.0|               Other|Other: explntion|           CORP|
|    2021|       1|       1160|       P|         Diaz Jr.|             Rube

In [17]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df5.select('OFFICECD').distinct().show()

+--------+
|OFFICECD|
+--------+
|       1|
+--------+



In [18]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
Expenditures_2017_df = df5.drop(*list2)
Expenditures_2017_df.show()

+--------+-----------+-----------------+------------------+--------------------+----------+-----+-------+------+--------------------+----------------+---------------+
|Election|CandidateID|CandidateLastName|CandidateFirstName|    ContributionType|      City|State|ZipCode|Amount|         PurposeCode|         Purpose|ContributerType|
+--------+-----------+-----------------+------------------+--------------------+----------+-----+-------+------+--------------------+----------------+---------------+
|    2021|       1160|         Diaz Jr.|             Ruben|                CORP|  New York|   NY|  10018|1300.0|         Fundraising|     Fundraising|           CORP|
|    2021|       1160|         Diaz Jr.|             Ruben|                CORP|  New York|   NY|  10018| 500.0|               Other|Other: explntion|           CORP|
|    2021|       1160|         Diaz Jr.|             Ruben|Limited Liability...|     Bronx|   NY|  10472| 350.0|               Other|Other: explntion|            LLC

In [ ]:
#Export to new CSV
#Expenditures_2017_df.toPandas().to_csv("Clean_Expenditures_2017.csv", header=True)

In [ ]:
# Store environmental variable
#from getpass import getpass
#Configure settings for RDS
#mode = "append"
#jdbc_url="jdbc:postgresql://database-final.cjdbapst3wjf.us-east-1.rds.amazonaws.com:5432/postgres"
#config = {"user":"postgres",
          #"password": "ADD PASSWORD WHEN READY",
          #"driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to active_user table in RDS
#Expenditures_2017_df.write.jdbc(url=jdbc_url, table='Expenditures_2017', mode=mode, properties=config)